# 开发 AI 应用

未来，AI 算法在日常生活中的应用将越来越广泛。例如，你可能想要在智能手机应用中包含图像分类器。为此，在整个应用架构中，你将使用一个用成百上千个图像训练过的深度学习模型。未来的软件开发很大一部分将是使用这些模型作为应用的常用部分。

在此项目中，你将训练一个图像分类器来识别不同的花卉品种。可以想象有这么一款手机应用，当你对着花卉拍摄时，它能够告诉你这朵花的名称。在实际操作中，你会训练此分类器，然后导出它以用在你的应用中。我们将使用[此数据集](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html)，其中包含 102 个花卉类别。你可以在下面查看几个示例。 

<img src='assets/Flowers.png' width=500px>

该项目分为多个步骤：

* 加载和预处理图像数据集
* 用数据集训练图像分类器
* 使用训练的分类器预测图像内容

我们将指导你完成每一步，你将用 Python 实现这些步骤。

完成此项目后，你将拥有一个可以用任何带标签图像的数据集进行训练的应用。你的网络将学习花卉，并成为一个命令行应用。但是，你对新技能的应用取决于你的想象力和构建数据集的精力。例如，想象有一款应用能够拍摄汽车，告诉你汽车的制造商和型号，然后查询关于该汽车的信息。构建你自己的数据集并开发一款新型应用吧。

首先，导入你所需的软件包。建议在代码开头导入所有软件包。当你创建此 notebook 时，如果发现你需要导入某个软件包，确保在开头导入该软件包。

In [1]:
# Imports here

%matplotlib inline
from torchvision import transforms, datasets, models
import torch
from torch import nn
from collections import OrderedDict
from torch import optim
import time
from tqdm import tqdm
import json
import copy
from torch.optim import lr_scheduler
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Make sure the model and data is always send to the correct device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 加载数据

在此项目中，你将使用 `torchvision` 加载数据（[文档](http://pytorch.org/docs/master/torchvision/transforms.html#)）。数据应该和此 notebook 一起包含在内，否则你可以[在此处下载数据](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz)。数据集分成了三部分：训练集、验证集和测试集。对于训练集，你需要变换数据，例如随机缩放、剪裁和翻转。这样有助于网络泛化，并带来更好的效果。你还需要确保将输入数据的大小调整为 224x224 像素，因为预训练的网络需要这么做。

验证集和测试集用于衡量模型对尚未见过的数据的预测效果。对此步骤，你不需要进行任何缩放或旋转变换，但是需要将图像剪裁到合适的大小。

对于所有三个数据集，你都需要将均值和标准差标准化到网络期望的结果。均值为 `[0.485, 0.456, 0.406]`，标准差为 `[0.229, 0.224, 0.225]`。这样使得每个颜色通道的值位于 -1 到 1 之间，而不是 0 到 1 之间。

In [2]:
data_dir = 'flowers'

trainkey = 'train'
validkey = 'valid'
testkey = 'test'

train_dir = data_dir + '/' + trainkey
valid_dir = data_dir + '/' + validkey
test_dir  = data_dir + '/' + testkey

dirs = { trainkey : train_dir
       , validkey : valid_dir
       , testkey  : test_dir}

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = {
    trainkey: transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    
    validkey: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),

    testkey : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

# TODO: Load the datasets with ImageFolder
image_datasets  = {x: datasets.ImageFolder(dirs[x], transform=data_transforms[x]) 
                 for x in [trainkey, validkey, testkey]}

dataset_sizes = {x: len(image_datasets[x]) 
                 for x in [trainkey, validkey, testkey]}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders   = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True) 
                 for x in [trainkey, validkey, testkey]}

clf_output = len(image_datasets[trainkey].classes)

### 标签映射

你还需要加载从类别标签到类别名称的映射。你可以在文件 `cat_to_name.json` 中找到此映射。它是一个 JSON 对象，可以使用 [`json` 模块](https://docs.python.org/2/library/json.html)读取它。这样可以获得一个从整数编码的类别到实际花卉名称的映射字典。

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# 构建和训练分类器

数据准备好后，就开始构建和训练分类器了。和往常一样，你应该使用 `torchvision.models` 中的某个预训练模型获取图像特征。使用这些特征构建和训练新的前馈分类器。

这部分将由你来完成。如果你想与他人讨论这部分，欢迎与你的同学讨论！你还可以在论坛上提问或在工作时间内咨询我们的课程经理和助教导师。

请参阅[审阅标准](https://review.udacity.com/#!/rubrics/1663/view)，了解如何成功地完成此部分。你需要执行以下操作：

* 加载[预训练的网络](http://pytorch.org/docs/master/torchvision/models.html)（如果你需要一个起点，推荐使用 VGG 网络，它简单易用）
* 使用 ReLU 激活函数和丢弃定义新的未训练前馈网络作为分类器
* 使用反向传播训练分类器层，并使用预训练的网络获取特征
* 跟踪验证集的损失和准确率，以确定最佳超参数

我们在下面为你留了一个空的单元格，但是你可以使用多个单元格。建议将问题拆分为更小的部分，并单独运行。检查确保每部分都达到预期效果，然后再完成下个部分。你可能会发现，当你实现每部分时，可能需要回去修改之前的代码，这很正常！

训练时，确保仅更新前馈网络的权重。如果一切构建正确的话，验证准确率应该能够超过 70%。确保尝试不同的超参数（学习速率、分类器中的单元、周期等），寻找最佳模型。保存这些超参数并用作项目下个部分的默认值。

In [5]:
# TODO: Build and train your network
model = models.vgg19(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

clf_input = model.classifier[0].in_features

clf_hidden = int(((clf_input - clf_output) / 6) + clf_output)

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(clf_input, clf_hidden)),
                          ('relu1', nn.ReLU()),
                          ('dp1',nn.Dropout(0.5)),
                          ('fc2', nn.Linear(clf_hidden, clf_output)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier = classifier

model.criterion = nn.NLLLoss()
model.optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
model.scheduler = lr_scheduler.StepLR(model.optimizer, step_size=4, gamma=0.1)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
100%|██████████| 574673361/574673361 [00:22<00:00, 25275295.56it/s]


In [6]:
def do_training(model, dataloaders, device, steps = 4) :

    model.to(device)
    best_model_sd = copy.deepcopy(model.state_dict())
    start = time.time()
    best_acc = 0
    
    for e in range(steps) :
        
        print('Step {}/{}'.format(e + 1, steps))
        print('-' * 11)
        
        for phase in [trainkey, validkey] :
            if phase == trainkey :
                model.train()
                model.scheduler.step()
            else :
                model.eval()
            
            running_loss     = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]) :
                inputs, labels = inputs.to(device), labels.to(device)
                
                model.optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == trainkey) :
                    outputs = model.forward(inputs)
                    loss    = model.criterion(outputs, labels)
                
                if phase == trainkey :
                    loss.backward()
                    model.optimizer.step()
                    
                running_loss += loss.item() * inputs.size(0)
                predictions = torch.max(outputs.data, 1)[1]
                running_corrects += torch.sum(predictions == labels.data)
            
            step_loss = running_loss / dataset_sizes[phase]
            step_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, step_loss, step_acc))
            
            if phase == validkey and step_acc >= best_acc :
                best_acc = step_acc
                best_model_sd = copy.deepcopy(model.state_dict())
    print("Total duration was {:.0f}m {:.0f}s.\n Best accuracy was {}".format((time.time() - start) / 60, (time.time() - start) % 60, best_acc))
    
    model.load_state_dict(best_model_sd)
    return model

In [7]:
model = do_training(model, dataloaders, device)

  0%|          | 0/410 [00:00<?, ?it/s]

Step 1/4
-----------


  0%|          | 0/52 [00:00<?, ?it/s]

train Loss: 3.5028 Acc: 0.2848


  0%|          | 0/410 [00:00<?, ?it/s]

...

valid Loss: 1.0237 Acc: 0.7506
Step 4/4
-----------


  0%|          | 0/52 [00:00<?, ?it/s]

train Loss: 2.1465 Acc: 0.4806


100%|██████████| 52/52 [00:27<00:00,  2.43it/s]

valid Loss: 0.9099 Acc: 0.7567
Total duration was 18m 58s.
 Best accuracy was 0.7567237163814181


## 测试网络

建议使用网络在训练或验证过程中从未见过的测试数据测试训练的网络。这样，可以很好地判断模型预测全新图像的效果。用网络预测测试图像，并测量准确率，就像验证过程一样。如果模型训练良好的话，你应该能够达到大约 70% 的准确率。

In [8]:
# TODO: Do validation on the test set
def do_evaluate(model, dataloader, device) :
    model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model.forward(images)
            predicted = torch.max(outputs.data, 1)[1]
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (100 * correct / total)

In [9]:
print(do_evaluate(model, dataloaders[testkey], device))

100%|██████████| 52/52 [00:28<00:00,  2.29it/s]

72.64957264957265


## 保存检查点

训练好网络后，保存模型，以便稍后加载它并进行预测。你可能还需要保存其他内容，例如从类别到索引的映射，索引是从某个图像数据集中获取的：`image_datasets['train'].class_to_idx`。你可以将其作为属性附加到模型上，这样稍后推理会更轻松。

In [11]:
# TODO: Save the checkpoint 
def save_checkpoint(model, architecture, image_dataset, filename):
    
    model.class_to_idx = image_dataset.class_to_idx
    
    dim_clf = {'input'  : model.classifier[0].in_features,
               'hidden' : model.classifier[0].out_features,
               'output' : model.classifier[3].out_features}
    
    model.cpu()
    torch.save({'architecture'     : architecture,
                'state_dict'       : model.classifier.state_dict(), 
                'class_to_idx'     : model.class_to_idx,
                'optim_state_dict' : model.optimizer.state_dict(),
                'sched_state_dict' : model.scheduler.state_dict(),
                'dim_clf'          : dim_clf}, 
                filename)

In [13]:
save_checkpoint(model, 'vgg19', image_datasets[trainkey], 'img_classifier.pth')

## 加载检查点

此刻，建议写一个可以加载检查点并重新构建模型的函数。这样的话，你可以回到此项目并继续完善它，而不用重新训练网络。

In [14]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath, device):
    checkpoint = torch.load(filepath, map_location=lambda storage, loc: storage)
    
    if   checkpoint['architecture'] == 'vgg16' :
        model = models.vgg16(pretrained=True)
    elif checkpoint['architecture'] == 'vgg19' :
        model = models.vgg19(pretrained=True)
    else :
        print("The architecture {} is not recognized, returned None instead".format(architecture))
        return None
    
    for param in model.parameters():
        param.requires_grad = False
    
    dim_clf = checkpoint['dim_clf']
    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(dim_clf['input'], dim_clf['hidden'])),
                          ('relu1', nn.ReLU()),
                          ('dp1',nn.Dropout(0.5)),
                          ('fc2', nn.Linear(dim_clf['hidden'], dim_clf['output'])),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
    
    model.classifier = classifier
    model.classifier.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    
    model.criterion = nn.NLLLoss()
    
    model.to(device)
    
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
    model.optimizer = optimizer
    model.optimizer.load_state_dict(checkpoint['optim_state_dict'])
    
    scheduler = lr_scheduler.StepLR(model.optimizer, step_size=4, gamma=0.1)
    model.scheduler = scheduler
    model.scheduler.load_state_dict(checkpoint['sched_state_dict'])
    
    return model

In [15]:
model = load_checkpoint('img_classifier.pth', device)

# 类别推理

现在，你需要写一个使用训练的网络进行推理的函数。即你将向网络中传入一个图像，并预测图像中的花卉类别。写一个叫做 `predict` 的函数，该函数会接受图像和模型，然后返回概率在前 $K$ 的类别及其概率。应该如下所示：

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

首先，你需要处理输入图像，使其可以用于你的网络。

## 图像处理

你需要使用 `PIL` 加载图像（[文档](https://pillow.readthedocs.io/en/latest/reference/Image.html)）。建议写一个函数来处理图像，使图像可以作为模型的输入。该函数应该按照训练的相同方式处理图像。

首先，调整图像大小，使最小的边为 256 像素，并保持宽高比。为此，可以使用 [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) 或 [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) 方法。然后，你需要从图像的中心裁剪出 224x224 的部分。

图像的颜色通道通常编码为整数 0-255，但是该模型要求值为浮点数 0-1。你需要变换值。使用 Numpy 数组最简单，你可以从 PIL 图像中获取，例如 `np_image = np.array(pil_image)`。

和之前一样，网络要求图像按照特定的方式标准化。均值应标准化为 `[0.485, 0.456, 0.406]`，标准差应标准化为 `[0.229, 0.224, 0.225]`。你需要用每个颜色通道减去均值，然后除以标准差。

最后，PyTorch 要求颜色通道为第一个维度，但是在 PIL 图像和 Numpy 数组中是第三个维度。你可以使用 [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html)对维度重新排序。颜色通道必须是第一个维度，并保持另外两个维度的顺序。

In [17]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    pil_img = Image.open(image_path)
    
    img = transforms.Resize(256)(pil_img)
    
    lmargin = (img.width-224) / 2   # left
    bmargin = (img.height-224) / 2  # bottom
    rmargin = lmargin + 224         # right
    tmargin = bmargin + 224         # top
    
    img = img.crop((lmargin, bmargin, rmargin, tmargin)) 
    
    img = np.array(img) / 255

    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225]) 
    img  = (img - mean) / std

    img = img.transpose((2, 0, 1))

    return img

要检查你的项目，可以使用以下函数来转换 PyTorch 张量并将其显示在  notebook 中。如果 `process_image` 函数可行，用该函数运行输出应该会返回原始图像（但是剪裁掉的部分除外）。

In [18]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image(r'/home/workspace/aipnd-project/flowers/test/28/Stemless-Gentian-Gentiana-acaulis-0555.jpg'))

## 类别预测

可以获得格式正确的图像后 

要获得前 $K$ 个值，在张量中使用 [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk)。该函数会返回前 `k` 个概率和对应的类别索引。你需要使用  `class_to_idx`（希望你将其添加到了模型中）将这些索引转换为实际类别标签，或者从用来加载数据的[ `ImageFolder`](https://pytorch.org/docs/master/torchvision/datasets.html?highlight=imagefolder#torchvision.datasets.ImageFolder)进行转换。确保颠倒字典

同样，此方法应该接受图像路径和模型检查点，并返回概率和类别。

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    
    img = process_image(image_path) 
    img = torch.from_numpy(img).type(torch.FloatTensor) 
    img = img.unsqueeze_(0)

    with torch.no_grad() :
        probabilities = torch.exp(model.forward(img))
    
    topk_prob, topk_idx = probabilities.topk(topk)
    topk_prob = topk_prob.tolist()[0]
    topk_idx  = topk_idx.tolist()[0]

    idx_to_class = {y:x for x,y in model.class_to_idx.items()}
    
    topk_flower = [cat_to_name[idx_to_class[cls]] for cls in topk_idx]

    return topk_prob, topk_flower

In [ ]:
# For this test case I downloaded two custom flowers from google 
# The first one is predicted correct
print(predict(r'/home/workspace/aipnd-project/flowers/test/28/Stemless-Gentian-Gentiana-acaulis-0555.jpg', model))
# The second one is incorrect, the correct flower is not within the top 5
print(predict(r'/home/workspace/aipnd-project/flowers/test/33/love-in-a-mist-persian-jewels-rose-red-flowers.jpg', model))

## 检查运行状况

你已经可以使用训练的模型做出预测，现在检查模型的性能如何。即使测试准确率很高，始终有必要检查是否存在明显的错误。使用 `matplotlib` 将前 5 个类别的概率以及输入图像绘制为条形图，应该如下所示：

<img src='assets/inference_example.png' width=300px>

你可以使用 `cat_to_name.json` 文件（应该之前已经在 notebook 中加载该文件）将类别整数编码转换为实际花卉名称。要将 PyTorch 张量显示为图像，请使用定义如下的 `imshow` 函数。

In [ ]:
# TODO: Display an image along with the top 5 classes
def show_prediction(image_path, model):
    
    plt.figure(figsize = (6,8))
    ax = plt.subplot(2,1,1)

    title = None
    for i in image_path.split('/') :
        if i.isdigit() :
            title = cat_to_name[i]
            break
            
    img = process_image(image_path)
    imshow(img, ax, title = title)
    probs, flowers = predict(image_path, model) 
    plt.subplot(2,1,2)
    sns.barplot(x=probs, y=flowers, color=sns.color_palette()[0]);

    return plt

In [ ]:
# Uploaded a picture that I found on google of a stemless gentian and evaluated the model. See results below.
show_prediction(r'/home/workspace/aipnd-project/flowers/test/28/Stemless-Gentian-Gentiana-acaulis-0555.jpg', model)